In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('hw08.ok')

# Homework 8: Confidence Intervals

**Helpful Resource:**
- [Python Reference](https://math-121-spring-21.github.io/python-reference.html): Cheat sheet of helpful array & table methods used in Math 121!

**Reading**: 
* [Estimation](https://www.inferentialthinking.com/chapters/13/estimation.html)

Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell to load the provided tests. Each time you start your server, you will need to execute this cell again to load the tests.

For all problems that you must write explanations and sentences for, you **must** provide your answer in the designated space. **Moreover, throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook!** For example, if you use `max_temperature` in your answer to one question, do not reassign it later on. Otherwise, you will fail tests that you thought you were passing previously!

In [ ]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

# These lines do some fancy plotting magic.",
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

from client.api.notebook import *
def new_save_notebook(self):
    """ Saves the current notebook by
        injecting JavaScript to save to .ipynb file.
    """
    try:
        from IPython.display import display, Javascript
    except ImportError:
        log.warning("Could not import IPython Display Function")
        print("Make sure to save your notebook before sending it to OK!")
        return

    if self.mode == "jupyter":
        display(Javascript('IPython.notebook.save_checkpoint();'))
        display(Javascript('IPython.notebook.save_notebook();'))
    elif self.mode == "jupyterlab":
        display(Javascript('document.querySelector(\'[data-command="docmanager:save"]\').click();'))   

    print('Saving notebook...', end=' ')

    ipynbs = [path for path in self.assignment.src
              if os.path.splitext(path)[1] == '.ipynb']
    # Wait for first .ipynb to save
    if ipynbs:
        if wait_for_save(ipynbs[0]):
            print("Saved '{}'.".format(ipynbs[0]))
        else:
            log.warning("Timed out waiting for IPython save")
            print("Could not automatically save \'{}\'".format(ipynbs[0]))
            print("Make sure your notebook"
                  " is correctly named and saved before submitting to OK!".format(ipynbs[0]))
            return False                
    else:
        print("No valid file sources found")
    return True

def wait_for_save(filename, timeout=600):
    """Waits for FILENAME to update, waiting up to TIMEOUT seconds.
    Returns True if a save was detected, and False otherwise.
    """
    modification_time = os.path.getmtime(filename)
    start_time = time.time()
    while time.time() < start_time + timeout:
        if (os.path.getmtime(filename) > modification_time and
            os.path.getsize(filename) > 0):
            return True
        time.sleep(0.2)
    return False

Notebook.save_notebook = new_save_notebook

ok = Notebook('hw08.ok')

Before continuing the assignment, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to okpy.org and flag the correct version. There will be another submit cell at the end of the assignment when you finish!

In [ ]:
_ = ok.submit()

## 1. Wheaton Restaurants


Ben and Frank are trying see what the best restaurant in Wheaton is. They survey 1500 Wheaton residents selected uniformly at random, and ask each person what restaurant is the best (*Note: this data is fabricated for the purposes of this homework*). The choices of are Ivy, Gia Mia, The Burger Social, and Los Burritos Tapatios. After compiling the results, Ben and Frank release the following percentages from their sample:

|Thai Restaurant  | Percentage|
|:------------:|:------------:|
|Ivy | 25% |
|Gia Mia | 53% |
|Burger Social | 14% |
|Los Burritos | 8% |

These percentages represent a uniform random sample of the population of Wheaton residents. We will attempt to estimate the corresponding *parameters*, or the percentage of the votes that each restaurant will receive from the entire population (the entire population is all Wheaton residents). We will use confidence intervals to compute a range of values that reflects the uncertainty of our estimates.

The table `votes` contains the results of the survey.

In [ ]:
# Just run this cell
votes = Table.read_table('wheaton_votes.csv')
votes

**Question 1.** Complete the function `one_resampled_percentage` below. It should return Gia Mia's **percentage** of votes after simulating one bootstrap sample of `tbl`. Reminder that a percentage is between 0 and 100.

**Note:** `tbl` will always be in the same format as `votes`.

<!--
BEGIN QUESTION
name: q1_1
manual: false
-->

In [ ]:
def one_resampled_percentage(tbl):
    ...

one_resampled_percentage(votes)

In [ ]:
ok.grade("q1_1");

**Question 2.** Complete the `percentages_in_resamples` function such that it simulates and returns an array of 2000 bootstrapped estimates of the percentage of voters who will vote for Gia Mia. You should use the `one_resampled_percentage` function you wrote above.

*Note:* There are no autograder tests for this question. 

<!--
BEGIN QUESTION
name: q1_2
manual: false
-->

In [ ]:
def percentages_in_resamples():
    percentage_gia = make_array()
    ...

In the following cell, we run the function you just defined, `percentages_in_resamples`, and create a histogram of the calculated statistic for the 2,000 bootstrap estimates of the percentage of voters who voted for Gia Mia. Based on what the original restaurant percentages were, does the graph seem reasonable? 

*Note: This might take a few seconds to run.*

In [ ]:
resampled_percentages = percentages_in_resamples()
Table().with_column('Estimated Percentage', resampled_percentages).hist("Estimated Percentage")

**Question 3.** Using the array `resampled_percentages`, find the values at the two edges of the middle 95% of the bootstrapped percentage estimates. (Compute the lower and upper ends of the interval, named `gia_lower_bound` and `gia_upper_bound`, respectively.)

*Hint:* If you are stuck on this question, try looking over chapter 13 of the textbook.

<!--
BEGIN QUESTION
name: q1_3
manual: false
-->

In [ ]:
gia_lower_bound = ...
gia_upper_bound = ...
print("Bootstrapped 95% confidence interval for the percentage of Gia Mia voters in the population: [{:f}, {:f}]".format(gia_lower_bound, gia_upper_bound))

In [ ]:
ok.grade("q1_3");

**Question 4.** The survey results seem to indicate that Gia Mia is beating all the other restaurants combined among voters. We would like to use confidence intervals to determine a range of likely values for Gia Mia's true lead over all the other restaurants combined. The calculation for Gia Mia's lead over Los Burritos, Burger Social and Ivy combined is:

$$\text{Gia Mia's % of the vote} - \text{(Ivy's % of the vote + Los's % of the vote + Burger Social's % of the vote)}$$

Define the function `one_resampled_difference` that returns **exactly one value** of Gia Mia's percentage lead over Los Burritos, Burger Social and Ivy combined from one bootstrap sample of `tbl`.

*Note: Given a table of votes, how can you figure out what percentage of the votes are for a certain restaurant?* **Be sure to use percentages, not proportions, for this question!**

<!--
BEGIN QUESTION
name: q1_4
manual: false
-->

In [ ]:
def one_resampled_difference(tbl):
    bootstrap = ...
    gia_percentage = ...
    lh_percentage = ...
    tt_percentage = ...
    tb_percentage = ...
    ...

In [ ]:
ok.grade("q1_4");

**Question 5.**
Write a function called `leads_in_resamples` that finds 2,000 bootstrapped estimates (the result of calling `one_resampled_difference`) of Gia Mia's lead over Los Burritos, Burger Social and Ivy combined. Plot a histogram of the resulting samples. 

**Note:** Gia Mia's lead can be negative.

Hint: If you see an error involving “NoneType”, consider what components a function needs to have. 

<!--
BEGIN QUESTION
name: q1_5
manual: true
-->
<!-- EXPORT TO PDF -->

In [ ]:
def leads_in_resamples():
    ...

sampled_leads = leads_in_resamples()
Table().with_column('Estimated Lead', sampled_leads).hist("Estimated Lead")

**Question 6.** Use the simulated data from Question 5 to compute an approximate 95% confidence interval for Imm Thai's true lead over Lucky House, Thai Temple, and Thai Basil combined.

<!--
BEGIN QUESTION
name: q1_6
manual: false
-->

In [ ]:
diff_lower_bound = ...
diff_upper_bound = ...
print("Bootstrapped 95% confidence interval for Gia Mia's true lead over Ivy, Los, and Burger Social combined: [{:f}, {:f}]".format(diff_lower_bound, diff_upper_bound))

In [ ]:
ok.grade("q1_6");

## 2. Interpreting Confidence Intervals


We computed the following 95% confidence interval for the percentage of Gia Mia voters: 

$$[50.53, 55.53]$$

(Your answer may have been a bit different due to randomness; that doesn't mean it was wrong!)

#### Question 1
Can we say there is a 95% probability that the interval $[50.53, 55.53]$ contains the true percentage of the population that votes for Gia Mia as the best Wheaton restaurant? Answer "yes" or "no" and explain your reasoning. 

*Note:* ambiguous answers using language like "sometimes" or "maybe" will not receive credit.

<!--
BEGIN QUESTION
name: q2_1
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 2**

Ben and Frank also created 70%, 90%, and 99% confidence intervals from the same sample, but we forgot to label which confidence interval represented which percentages! Match each confidence level (70%, 90%, 99%) with its corresponding interval in the cell below (e.g. __ % CI: [52.1, 54] $\rightarrow$ replace the blank with one of the three confidence levels). **Then**, explain your thought process and how you came up with your answers.

The intervals are below:

* [50.03, 55.94]
* [52.1, 54]
* [50.97, 54.99]


<!--
BEGIN QUESTION
name: q2_2
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

#### Question 3
Suppose we produced 6,000 new samples (each one a uniform random sample of 1,500 voters) from the population and created a 95% confidence interval from each one. Roughly how many of those 6,000 intervals do you expect will actually contain the true percentage of the population?

Assign your answer to `true_percentage_intervals`.

<!--
BEGIN QUESTION
name: q2_3
manual: false
-->

In [ ]:
true_percentage_intervals = ...

In [ ]:
ok.grade("q2_3");

Recall the second bootstrap confidence interval you created, which estimated Gia Mia's lead over Ivy, Los Burritos, and Burger Social combined. Among
voters in the sample, Gia Mia's lead was 6%. The bootstrapped 95% confidence interval for the true lead (in the population of all voters) was

$$[1.2, 11.2]$$

(again, this may be slightly different from yours!) Suppose we are interested in testing a simple yes-or-no question:

> "Is the percentage of votes for Gia Mia tied with the percentage of votes for Ivy, Los Burritos, and Burger Social combined?"

Our null hypothesis is that the percentages are equal, or equivalently, that Gia Mia's lead is exactly 0. Our alternative hypothesis is that Gia Mias's lead is not equal to 0.  In the questions below, don't compute any confidence interval yourself - use only the 95% confidence interval above.


**Question 4** 

Say we use a 5% P-value cutoff.  Do we reject the null, fail to reject the null, or are we unable to tell using our confidence interval?

Assign `restaurants_tied` to the number corresponding to the correct answer.

1. Reject the null / Data is consistent with the alternative hypothesis
2. Fail to reject the null / Data is consistent with the null hypothesis
3. Unable to tell using our confidence interval

*Hint:* Consider the relationship between the p-value cutoff and confidence. If you're confused, take a look at [this chapter](https://www.inferentialthinking.com/chapters/13/4/using-confidence-intervals.html) of the textbook.

<!--
BEGIN QUESTION
name: q2_4
manual: false
-->

In [ ]:
restaurants_tied = ...

In [ ]:
ok.grade("q2_4");

#### Question 5
What if, instead, we use a P-value cutoff of 1%? Do we reject the null, fail to reject the null, or are we unable to tell using our confidence interval?

Assign `cutoff_one_percent` to the number corresponding to the correct answer.

1. Reject the null / Data is consistent with the alternative hypothesis
2. Fail to reject the null / Data is consistent with the null hypothesis
3. Unable to tell using our confidence interval

<!--
BEGIN QUESTION
name: q2_5
manual: false
-->

In [ ]:
cutoff_one_percent = ...

In [ ]:
ok.grade("q2_5");

#### Question 6
What if we use a P-value cutoff of 10%? Do we reject, fail to reject, or are we unable to tell using our confidence interval?

Assign `cutoff_ten_percent` to the number corresponding to the correct answer.

1. Reject the null / Data is consistent with the alternative hypothesis
2. Fail to reject the null / Data is consistent with the null hypothesis
3. Unable to tell using our confidence interval

<!--
BEGIN QUESTION
name: q2_6
manual: false
-->

In [ ]:
cutoff_ten_percent = ...

In [ ]:
ok.grade("q2_6");

## 3. Submission


Once you're finished, select "Save and Checkpoint" in the File menu and then execute the `submit` cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to [okpy.org](https://okpy.org/) and flag the correct version. To do so, go to the website, click on this assignment, and find the version you would like to have graded. There should be an option to flag that submission for grading!

In [ ]:
_ = ok.submit()

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q') and len(q) <= 10]
print("Finished running all tests.")